In [1]:
from nlp import load_dataset
from sklearn.metrics import accuracy_score
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments

### Part 1: Data Loaders
(Using nlp scripts)

In [2]:
train_dataset, test_dataset = load_dataset('emo_data.py', split = ['train', 'test'])

In [3]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 4)
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [4]:
def tokenize(batch):
    return tokenizer(batch['text'], padding = True, truncation = True)

In [5]:
train_dataset = train_dataset.map(tokenize, batched = True, batch_size = len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched = True, batch_size = len(train_dataset))
train_dataset.set_format('torch', columns = ['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns = ['input_ids', 'attention_mask', 'label'])

In [6]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc
    }

### Part 2: Fine Tune

In [7]:
training_args = TrainingArguments(
    output_dir = './results',
    num_train_epochs = 1,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 64,
    warmup_steps = 500,
    weight_decay = 0.01,
    evaluate_during_training = True,
    logging_dir = './logs',
)

trainer = Trainer(
    model = model,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

In [8]:
trainer.train()

/opt/conda/conda-bld/pytorch_1587428094786/work/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.
/home/tanmay/anaconda3/envs/torch/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


TrainOutput(global_step=1885, training_loss=0.45074596978941395)

In [9]:
trainer.evaluate()

{'eval_loss': 0.36143595932972156,
 'eval_accuracy': 0.8751134507170085,
 'epoch': 1.0}

In [12]:
trainer.save_model("../models/emo-fine-tune/")

### Part 3: Pipeline

In [13]:
from transformers import pipeline